In [1]:
import rospy
import pickle
import moveit_commander
from sensor_msgs.msg import JointState


In [8]:
import moveit_msgs.msg
rospy.init_node('notebook')
state_pub = rospy.Publisher("notebook_state", moveit_msgs.msg.DisplayRobotState, queue_size=1)
traj_pub = rospy.Publisher("notebook_traj", moveit_msgs.msg.DisplayTrajectory, queue_size=1)

In [2]:
with open("/home/hri25-group3/ros/src/safety_perception/joint_snapshot1.pkl", "rb") as f:
        data = pickle.load(f)

In [16]:
data[0]

{'t': 1761213852.0178509,
 'names': ['arm_shoulder_pan_joint',
  'arm_shoulder_lift_joint',
  'arm_elbow_joint',
  'arm_wrist_1_joint',
  'arm_wrist_2_joint',
  'arm_wrist_3_joint'],
 'q': [-3.1371305624591272,
  -2.5746733150877894,
  -1.473508358001709,
  -0.6813049477389832,
  1.5681782960891724,
  -0.3417566458331507]}

In [10]:
s = data[5]
print(s['names'])

['arm_shoulder_pan_joint', 'arm_shoulder_lift_joint', 'arm_elbow_joint', 'arm_wrist_1_joint', 'arm_wrist_2_joint', 'arm_wrist_3_joint']


In [12]:
ds = moveit_msgs.msg.DisplayRobotState()
ds.state.joint_state.name = s['names']
ds.state.joint_state.position = s['q']
state_pub.publish(ds)


In [ ]:
move = moveit_commander.MoveGroupCommander('arm')
move.set_max_velocity_scaling_factor(0.5)

move.set_named_target('ready')
move.go()


for i in range (len(data)-1):
    #start
    js1= JointState()
    js1.name=data[i]['names']
    js1.position=data[i]['q']
    move.set_joint_value_target(js1)
    move.set_max_velocity_scaling_factor(0.5)
    move.go()
    j=i+1
    for j in range (len(data)-1):
        #end
        js2= JointState()
        js2.name=data[i]['names']
        js2.position=data[i]['q']
        move.set_joint_value_target(js2)
        move.set_max_velocity_scaling_factor(0.5)
        move.go()
        #go back to start
        move.set_joint_value_target(js1)
        move.set_max_velocity_scaling_factor(0.5)
        move.go()
        
            


range(0, 11)

In [4]:
with open("/home/hri25-group3/ros/src/safety_perception/good_pairs_fast.pkl", "rb") as f:
        plans = pickle.load(f)

In [6]:
plans[0,1]

joint_trajectory: 
  header: 
    seq: 0
    stamp: 
      secs: 0
      nsecs:         0
    frame_id: "base"
  joint_names: 
    - arm_shoulder_pan_joint
    - arm_shoulder_lift_joint
    - arm_elbow_joint
    - arm_wrist_1_joint
    - arm_wrist_2_joint
    - arm_wrist_3_joint
  points: 
    - 
      positions: [-3.1371927897082728, -2.5750514469542445, -1.4729422330856323, -0.6812046331218262, 1.568124771118164, -0.3418238798724573]
      velocities: [0.03255632944235696, 0.006502850424413667, 0.055635497965112704, -0.06283185307179588, 1.0580283829659567e-05, 3.1915709889456236e-06]
      accelerations: [3.2556329442356953, 0.6502850424413665, 5.5635497965112695, -6.283185307179587, 0.0010580283829659566, 0.0003191570988945623]
      effort: []
      time_from_start: 
        secs: 0
        nsecs:         0
    - 
      positions: [-3.120914624987094, -2.571800021742038, -1.4451244841030761, -0.7126205596577242, 1.5681300612600788, -0.34182228408696286]
      velocities: [0.325563

In [17]:
dtraj = moveit_msgs.msg.DisplayTrajectory()
rtraj = moveit_msgs.msg.RobotTrajectory()
rtraj.joint_trajectory = plans[0,5].joint_trajectory
dtraj.trajectory.append(rtraj)
traj_pub.publish(dtraj)

In [14]:
plans.keys()

dict_keys([(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 8), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (1, 8), (2, 3), (2, 5), (2, 8), (4, 5), (4, 8)])